In [ ]:
import sys, os, math
import statistics
import numpy as np
import pandas as pd
import pickle as pkl

import umap

sys.path.append('../ml_pipeline')
import label_converter

# import functions instead of having them in here to keep the notebook *much* shorter
sys.path.append('functions')
import confusion_matrix
import roc_curve
import image_excerpt
import beeswarm
import image_bytestream
import bokeh_wrapper
import umap_embedding
import entropy_plot
import sc_occlusion
import load_data
import prediction_barplot


fontsize=12

# Define analysis.
To analyze cross-validation, enter the path of the first fold folder. Each folder for one fold should contain ```_x``` at the end of the name, allowing the algorithm to identify the fold. Example:
```
result_folder
    abmil_0
    abmil_1
    abmil_2
    ...
```

The proper value to set for ```result_folder_path``` would be: ```result_folder/abmil_0``` and the notebook will recognize cross validation.

In [ ]:
fig_export_path = 'output'
result_folder_path= '/storage/groups/qscd01/projects/aml_mil_hehr/final_results/pub_0'
dataset_folder = '/storage/groups/qscd01/datasets/210526_mll_mil_pseudonymized'
feature_prefix = 'fnl34_'

# Identify cross valdation & load all the data
This process loads all feature vectors and prepares all the data for further display, so it will take a while.

In [ ]:
# identify all folders with cross validation
folders_cv_truncated = os.path.basename(result_folder_path)[:-1]
folders_cv_available = [f for f in os.listdir('{}/..'.format(result_folder_path)) if folders_cv_truncated == f[:-1]]
print("Found {} folder{}, loading data.".format(len(folders_cv_available), 's' if len(folders_cv_available) > 1 else ''))

data = load_data.load_dataframes(folders_cv_available, 
                    os.path.dirname(result_folder_path),
                    feature_prefix,
                    dataset_folder)

lbl_conv_obj, patient_df, sc_df = data
print("Classification labels: {}".format(list(lbl_conv_obj.df.true_lbl)))
print("Annotated patients: ", list(sc_df.loc[~sc_df['mll_annotation'].isna()].index.unique()))

In [ ]:
# calculate some patient data: total images, images per patient mean + sd
print("Total images: ", len(sc_df))
print("Images per patient (mean): ", sc_df.index.value_counts().mean())
print("Images per patient (std): ", sc_df.index.value_counts().std())

# Dataset: classification performance

### Confusion matrix
Analyze overall classification performance in 5-fold cross-vaildation.

In [ ]:
reorder=['AML-PML-RARA',
        'AML-NPM1',
        'AML-CBFB-MYH11',
        'AML-RUNX1-RUNX1T1',
        'SCD']

confusion_data = load_data.extract_confusion_matrix(patient_df, lbl_conv_obj)
confusion_matrix.show_pred_mtrx(pred_mtrx = confusion_data,
                                class_conversion = lbl_conv_obj.df,
                               reorder=reorder, fig_size=(8.1,4.5), sc_df=sc_df,
                               path_save=os.path.join(fig_export_path, 'confusion_matrix.svg'))

print("Overall accuracy: {}".format(confusion_matrix.get_accuracy(confusion_data)))
confusion_matrix.get_classwise_values_as_df(confusion_data, lbl_conv_obj).round(2)

In [ ]:
# fold-wise performance
full_confusion_data = {}
for fold in sorted(patient_df.fold.unique()):
    patient_df_filtered = patient_df.loc[patient_df['fold'] == fold]
    confusion_data = load_data.extract_confusion_matrix(patient_df_filtered, lbl_conv_obj)
    full_confusion_data[fold] = confusion_data

confusion_matrix.get_fold_statistics(full_confusion_data, lbl_conv_obj).round(2)

### ROC curves
Plot ROC curves for sensitivity/specificity and precision/recall for any (combination of) labels

In [ ]:
true_label = ['AML-PML-RARA']   #can also contain a list of multiple labels
roc_curve.plot(patient_df, true_label)

### Entropy plots
Plot patient entropy (divergence of prediction values)

In [ ]:
entropy_plot.entropy_plot(patient_df)
# entropy_vs_myb(patient_df)

### APL misclassifications

In [ ]:
patient_df.loc[(patient_df['gt_label'] == 'AML-PML-RARA') & ~(patient_df['pred_lbl'] == 'AML-PML-RARA')]

In [ ]:
patient_df.loc[patient_df['gt_label'] == patient_df['pred_lbl']].sample(1)

# Patients: algorithm performance
### Define a patient
Put in a patient ID to look at the predictions

In [ ]:
patient_id = 'UWF'
patient_df.loc[patient_id]

#'BHS', 'UGU', 'PKC', 'SBY', 'UVT'

In [ ]:
path_save=os.path.join(fig_export_path, 'barplots', 'prediction_{}.svg'.format(patient_id))
prediction_barplot.plot(patient_df.loc[patient_id], reorder=reorder, path_save=path_save)

### Show random set of images

In [ ]:
filtered_sc_data = sc_df.loc[patient_id]
image_excerpt.plot(filtered_sc_data[::3], show_scalebar=True)

### Show images ordered by relevance

In [ ]:
predicted_class = patient_df.loc[patient_id].pred_lbl
filtered_sc_data = sc_df.loc[patient_id].copy()
filtered_sc_data = filtered_sc_data.sort_values(by=load_data.get_softmax_attention_column(predicted_class), ascending=False)

# sample X images in a representative fashion
sample_count = 96
show_every = len(filtered_sc_data)/sample_count
filtered_sc_data['tmp'] = range(len(filtered_sc_data))
filtered_sc_data = filtered_sc_data.loc[filtered_sc_data['tmp']%show_every < 1] 

path_save=os.path.join(fig_export_path, 'image_excerpts', 'sample_sorted_{}.svg'.format(patient_id))
image_excerpt.plot(filtered_sc_data[::], show_scalebar=False, cols=12, show_coordinates=True, path_save=path_save)

### Show swarmplot
Swarmplot is interactive and shows cells upon mouseover. Classes of cells can be excluded by clicking the corresponding label in the legend on the right. This cell automatically stores a vector graphic in the folder ```output/swarmplots```, and calculates the distribution of cells in each quartile (see dataframe below interactive figure)

In [ ]:
predicted_class = patient_df.loc[patient_id].pred_lbl
true_class = patient_df.loc[patient_id].gt_label
filtered_sc_data = sc_df.loc[patient_id]
data_with_mappings = image_bytestream.map_images_to_dataframe(filtered_sc_data)
data_with_mappings_and_coordinates, xlim, ylim = beeswarm.beeswarm_coordinates(data_with_mappings, 
                                                                               val_col=load_data.get_softmax_attention_column(predicted_class))
data_with_mappings_and_coordinates['color_values'] = data_with_mappings_and_coordinates['mll_annotation'].fillna('cell')


# show interactive plot
bokeh_wrapper.multi_swarmplot(data_with_mappings_and_coordinates, 
          title='Swarmplot for patient {}, prediction: {}, true label: {}'.format(patient_id, predicted_class, true_class), 
          xlim=xlim, ylim=ylim)

# save interactive plot
path_save = os.path.join(fig_export_path, 'swarmplots', patient_id +"_swarmplot_interactive.html")
bokeh_wrapper.multi_swarmplot(data_with_mappings_and_coordinates, 
          title='Swarmplot for patient {}, prediction: {}, true label: {}'.format(patient_id, predicted_class, true_class), 
          xlim=xlim, ylim=ylim, path_save=path_save)

# define which images should be highlighted for the annotated patients
highlight_idx = {
    'PKC':[46, 286, 247, 186, 386, 198, 1],
    'UGU':[372, 197, 376, 73, 144, 174],
    'SBY':[340, 386, 220, 302, 87, 358],
    'UVT':[34, 454, 358, 207, 384, 131, 95],
    'BHS':[49, 200, 365, 69, 219, 184, 39]
}

get_highlight = lambda x: None if not x in highlight_idx.keys() else highlight_idx[x]
path_save = os.path.join(fig_export_path, 'swarmplots', patient_id +"_swarmplot.svg")
bokeh_wrapper.export_swarmplot(data_with_mappings_and_coordinates, 
          title='Swarmplot for patient {}, prediction: {}, true label: {}'.format(patient_id, predicted_class, true_class), 
          xlim=xlim, ylim=ylim, highlight_idx=get_highlight(patient_id), path_save=path_save)

# print quantiles and distribution of cells
quantiles, borders = bokeh_wrapper.calculate_cells_in_quantiles(data_with_mappings_and_coordinates, 
                                                       load_data.get_softmax_attention_column(predicted_class), 
                                                       group_index=True,
                                                      sort_by_percentage=False)
path_save = os.path.join(fig_export_path, 'swarmplots', patient_id +"_swarmplot_pie.svg")
bokeh_wrapper.plot_piechart(data_with_mappings_and_coordinates, load_data.get_softmax_attention_column(predicted_class), scale_factor=0.3, path_save=path_save)

# UMAP embedding

The interactive UMAP figures require quite a lot of RAM and computing power. To calculate the occlusion values, the use of CUDA-capable GPUs is highly recommended and will greatly speed up the process.

1. Calculate or load the UMAP embedding (not necessary, if an old gzip file should be loaded)

In [ ]:
# sample cells randomly for embedding
fold_filter = 0
sc_umap_sample = sc_df.loc[sc_df['fold'] == fold_filter].sample(frac=1, random_state=1).copy()

sc_df_umap = umap_embedding.select_embedding(sc_umap_sample)

2. Load all additional image data and calculate occlusion values. Either save the dataframe, or skip the data preparation process and load an old one. For this, pyarrow is required (https://anaconda.org/conda-forge/pyarrow)

In [ ]:
load=input("Load data?    (y/n): ")=='y'
save=input("Save results? (y/n): ")=='y'

load_dir = 'suppl_data/dataframe_saves'
if not load:
    # prepare SC data for umap embedding: add all annotated patients
    sc_umap_annotated = sc_df.loc[~sc_df['mll_annotation'].isna() & ~(sc_df['fold'] == fold_filter)]
    sc_umap_annotated = umap_embedding.embed_new_data(sc_umap_annotated)
    sc_prepared = pd.concat([sc_df_umap, sc_umap_annotated], axis=0)
    
    # load single cell images
#     sc_prepared = image_bytestream.map_images_to_dataframe(sc_prepared)
    
    # calculate occlusion values
    sc_prepared = sc_occlusion.calculate_change_on_occlusion(sc_prepared, result_folder_path, 
                                               folders_cv_available, feature_prefix, lbl_conv_obj)
else:
    
    print("Found data: ")
    print(os.listdir(load_dir))
    load_name = input("Enter dataframe to load: ")
    sc_prepared = pd.read_parquet(os.path.join(load_dir, load_name))
    
if save:
    save_name = input("Save dataframe as: ")
    sc_prepared.to_parquet(os.path.join(load_dir, save_name))

print("Operation complete. ")

##### Categorical umap

In [ ]:
bokeh_wrapper.umap(sc_prepared)

##### Occlusion UMAP

In [ ]:
occlusion_for_class = 'AML-PML-RARA'
bokeh_wrapper.umap(sc_prepared, title="Occlusion-UMAP", data_column="occlusion_{}".format(occlusion_for_class))

##### Attention UMAP

In [ ]:
attention_for_class = 'AML-RUNX1-RUNX1T1'
bokeh_wrapper.umap(sc_prepared, title="UMAP", legend_header="Annotated cell type", 
                   data_column=load_data.get_raw_attention_column(occlusion_for_class), grayscatter=True)

##### Export all UMAPs

In [ ]:
dtype='png'

highlight_in_scatter = [
    17485,   # atypical promyelocyte
    16944,   # myeloblast
    16978,   # monocyte
    17189,   # NGS
    17536,   # normo
    17476,   # metamyelocyte
    3777,    # NGB
    16696,   # EOS
    16147,   # smudge
    1781,    # lymph
    16180,   # LGL
]

sc_prepared['highlight'] = [False]*len(sc_prepared)
sc_prepared.loc[sc_prepared['index'].isin(highlight_in_scatter), 'highlight'] = [True]*len(highlight_in_scatter)

path_save = os.path.join(fig_export_path, 'umaps_categorical', "umap_full_fold.{}".format('png'))
bokeh_wrapper.export_umap(sc_prepared, data_column='mll_annotation', legend_capt='Annotated cell class', 
                  highlight=True, zorder_adapt_by_color=True, grayscatter=True, dotsize=35, path_save=path_save)

sc_prepared['highlight'] = [False]*len(sc_prepared)

# categorical umaps for every patient
annotated_indices = sc_prepared.loc[~sc_prepared['mll_annotation'].isna()].index.unique()
for idx in annotated_indices:
    subframe = sc_prepared.copy()
    annotations_stash = subframe.loc[idx, 'mll_annotation']
    subframe['mll_annotation'] = [None]*len(subframe)
    subframe.loc[idx, 'mll_annotation'] = annotations_stash
    path_save = os.path.join(fig_export_path, 'umaps_categorical', "umap_idx_{}.{}".format(idx, dtype))

    bokeh_wrapper.export_umap(subframe, data_column='mll_annotation', legend_capt='Annotated cell class', 
                    highlight=False, zorder_adapt_by_color=True, grayscatter=True, dotsize=35, path_save=path_save)

for class_lbl in list(lbl_conv_obj.df.true_lbl):
    
    path_save = os.path.join(fig_export_path, 'umaps_occlusion', "occlusion_{}.{}".format(class_lbl, dtype))
    bokeh_wrapper.export_umap(sc_prepared, data_column="occlusion_{}".format(class_lbl), legend_capt='Annotated cell class', 
                  highlight=False, zorder_adapt_by_color=False, grayscatter=True, dotsize=10, path_save=path_save)
    
    path_save = os.path.join(fig_export_path, 'umaps_attention', "attention_{}.{}".format(class_lbl, dtype))
    bokeh_wrapper.export_umap(sc_prepared, data_column=load_data.get_raw_attention_column(class_lbl), legend_capt='Annotated cell class', 
                  highlight=False, zorder_adapt_by_color=False, grayscatter=True, dotsize=10, path_save=path_save)